<a href="https://colab.research.google.com/github/lphansen/MFRSuite/blob/main/MFRSuite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MFRSuite

In [ ]:
!git clone https://github.com/lphansen/MFRSuite
!cd /content/MFRSuite/mfrSuite && bash install.sh

In [ ]:
import mfr.modelSoln as m
import numpy as np
import argparse
import time
import os
import json
import pickle
import itertools

# Production

Capital Evolution

$$
d k_{j, t}=k_{j, t}\left[\left(g_t+\iota_{j, t}-\delta\right) d t+\sqrt{s_t} \sigma \cdot d Z_t+\sqrt{\varsigma_t} d Z_{j, t}\right]
$$

- $g_t$ is exogenous productivity growth, $\iota_{j . t}$ denotes the investment rate, $\delta$ is depreciation



 Exogenous States
 
$$
\begin{aligned}
d g_t & =\lambda_g\left(\bar{g}-g_t\right) d t+\sqrt{s_t} \sigma_g \cdot d Z_t \\
d s_t & =\lambda_s\left(\bar{s}-s_t\right) d t+\sqrt{s_t} \sigma_s \cdot d Z_t \\
d \varsigma_t & =\lambda_{\varsigma}\left(\bar{\varsigma}-\varsigma_t\right) d t+\sqrt{\varsigma_t} \sigma_{\varsigma} \cdot d Z_t
\end{aligned}
$$



# Financial Market

Capital Price

$$
d q_t=q_t\left[\mu_{q, t} d t+\sigma_{q, t} \cdot d Z_t\right]
$$


SDF

$$
d S_t=-S_t\left[r_t d t+\pi_t \cdot d Z_t\right]
$$

- $r$ is the short-term interest rate, and $\pi$ denotes the $d \times 1$ vector of risk prices associated with $Z$.



 Return on Capital

$$
d R_{j, t}^k:=\frac{a_j k_{j, t}-\Phi\left(\iota_{j, t}\right) k_{j, t}}{q_t k_{j, t}} d t+\frac{d\left(q_t k_{j, t}\right)}{q_t k_{j, t}}
$$

- dividend yields $\frac{a_j-\Phi\left(\iota_{j, t}\right)}{q_t}$
- capital gain $\frac{d\left(q_t k_{j, t}\right)}{q_t k_{j, t}}$
  - Adjustment costs $\Phi(x)=\phi^{-1}[\exp (\phi x)-1]$

$$
d R_{j, t}^k=\underbrace{\left[\frac{a_j-\Phi\left(\iota_{j, t}\right)}{q_t}+\iota_{j, t}-\delta+g_t+\mu_{q, t}+\sigma_{K, t} \cdot \sigma_{q, t}\right]}_{:=\mu_{R, j, t}} d t+\underbrace{\left[\sigma_{K, t}+\sigma_{q, t}\right]}_{:=\sigma_{R, t}} \cdot d Z_t+\sqrt{\varsigma_t} d Z_{j, t}
$$





# Households

Preference

$$
U_{j, t}=\mathbb{E}\left[\int_0^{\infty} \varphi_j\left(c_{j, t+s}, U_{j, t+s}\right) d s \mid \mathcal{F}_t\right]
$$

- where $\varphi_j(c, U):=\rho_j \frac{1-\gamma_j}{1-\psi_j} U\left(c^{1-\psi_j}\left[\left(1-\gamma_j\right) U\right]^{-\frac{1-\psi_j}{1-\gamma_j}}-1\right)$



Dynamic Budget Constraints

$$
\frac{d n_{j, t}}{n_{j, t}}=\left(\mu_{n_j, t}-c_{j, t} / n_{j, t}\right) d t+\sigma_{n_j, t} \cdot d Z_t+\tilde{\sigma}_{n_j, t} d Z_{j, t}
$$

- where

  $$
  \begin{aligned}
  \mu_{n_j, t} & :=r_t+\left(q_t k_{j, t} / n_{j, t}\right)\left(\mu_{R, j, t}-r_t\right)-\left(1-\chi_{j, t}\right)\left(q_t k_{j, t} / n_{j, t}\right) \sigma_{R, t} \cdot \pi_t+\theta_{j, t} \cdot \pi_t \\
  \sigma_{n_j, t} & :=\chi_{j, t}\left(q_t k_{j, t} / n_{j, t}\right) \sigma_{R, t}+\theta_{j, t} \\
  \tilde{\sigma}_{n_j, t} & :=\chi_{j, t}\left(q_t k_{j, t} / n_{j, t}\right) \sqrt{\varsigma_t}
  \end{aligned}
  $$

  - which comes from

    $$
    \begin{aligned}
    \mu_{n_j, t}&=\frac{q_tk_{j,t}\mu_{R, j, t}-(1-\chi_{j,t})q_tk_{j,t}\left(r_t+\sigma_{R, t} \cdot \pi_t\right)-(\chi_{j, t} q_t k_{j, t}-n_{j, t})r_t+\theta_{j, t}n_{j,t}\pi_t}{n_{j, t}}\\
    &=r_t+\left(q_t k_{j, t} / n_{j, t}\right)\left(\mu_{R, j, t}-r_t\right)-\left(1-\chi_{j, t}\right)\left(q_t k_{j, t} / n_{j, t}\right) \sigma_{R, t} \cdot \pi_t+\theta_{j, t} \cdot \pi_t 
    \end{aligned}
    $$

- total asset $q_tk_{j,t}$

- equity issuances $(1-\chi_{j,t})q_tk_{j,t}$,

  - equity dividend $\left(r_t+\sigma_{R, t} \cdot \pi_t\right) d t+\sigma_{R, t} \cdot d Z_t+\sqrt{\varsigma_t} d Z_{j, t}$

- risk-free short debt $\chi_{j, t} q_t k_{j, t}-n_{j, t}$, 

  - risk-free rate $r_tdt$

- excess return on capital

  - $\left(q_t k_{j, t} / n_{j, t}\right)\left(\mu_{R, j, t}-r_t\right)$

- excess return compensation to equity investors

  - $\left(1-\chi_{j, t}\right)\left(q_t k_{j, t} / n_{j, t}\right) \sigma_{R, t} \cdot \pi_t$

- Derivative hedging positions $\theta_{j, t} n_{j, t}$

  - payout per unit $\pi_t d t+d Z_t$



Equity Constraint

$$
\chi_{j, t} \geq \underline{\chi}_j
$$


Hedging Constraint

$$
\theta_{i t} \in \Theta_j
$$


Borrowing Constraint

$$
\frac{\chi_{j, t} q_t k_{j, t}}{n_{j, t}} \sigma_{R, t} \leq \bar{\beta}_j
$$



# Market Clearing

Goods market clearing

$$
\int_{\mathbb{J} \in \mathbb{J}_h} c_{j, t} d j=\int_{\mathbb{J}_e \cup \mathbb{J}_h}\left(a_j-\Phi\left(\iota_{j, t}\right)\right) k_{j, t} d j .
$$

Capital market clearing

$$
\int_{\mathbb{J}_e \cup \mathbb{J}_h} k_{j, t} d j=K_t 
$$

Equity market clearing

$$
\int_{\mathbb{J}_e \cup \mathbb{J}_h}\left(1-\chi_{j, t}\right) q_t k_{j, t} \sigma_{R, t} d j=\int_{\mathbb{J}_e \cup \mathbb{J}_h} \theta_{j, t} n_{j, t} d j .
$$

Bond market clearing
$$
\int_{\mathbb{J}_e \cup \mathbb{J}_h}\left(q_t k_{j, t}-n_{j, t}\right) d j=0
$$



# HJB

HJB equation

$$
\begin{aligned}
0=\max \frac{\rho}{1-\psi}\left[(c / n)^{1-\psi} \xi^{\psi-1}-1\right]+\mu_n-c / n+\mu_X \cdot \partial_X \ln \xi-\frac{\gamma}{2}\left(\left\|\sigma_n\right\|^2+\tilde{\sigma}_n^2\right) \\
+(1-\gamma) \sigma_n \cdot \sigma_X^{\prime} \partial_X \ln \xi+\frac{1}{2}\left[\operatorname{tr}\left(\sigma_X^{\prime} \partial_{X X^{\prime}} \ln \xi \sigma_X\right)+(1-\gamma)\left\|\sigma_X^{\prime} \partial_X \ln \xi\right\|^2\right]
\end{aligned}
$$


FOC on investment

$$
\begin{aligned}
\Phi(\iota) &=\phi^{-1}[\exp (\phi \iota)-1]\\
\iota(q) & =\phi^{-1} \log (q) \\
\Phi(\iota(q)) & =\phi^{-1}(q-1) .
\end{aligned}
$$


FOC on consumption

$$
c_{j, t}=\rho_j^{1 / \psi_j} \xi_{j, t}^{1-1 / \psi_j} n_{j, t}
$$



HJB equation with optimized investment and consumption

$$
\begin{aligned}
0=\max & \frac{\psi}{1-\psi} \rho^{1 / \psi} \xi^{1-1 / \psi}-\frac{\rho}{1-\psi}+\mu_n-\frac{\gamma}{2}\left(\left\|\sigma_n\right\|^2+\tilde{\sigma}_n^2\right)+\left[\mu_X+(1-\gamma) \sigma_X \sigma_n\right] \cdot \partial_X \ln \xi \\+&\frac{1}{2}\left[\operatorname{tr}\left(\sigma_X^{\prime} \partial_{X X^{\prime}} \ln \xi \sigma_X\right)+(1-\gamma)\left\|\sigma_X^{\prime} \partial_X \ln \xi\right\|^2\right]
\end{aligned}
$$

- where

$$
\begin{aligned}
\mu_{n_j, t} & :=r_t+\left(q_t k_{j, t} / n_{j, t}\right)\left(\mu_{R, j, t}-r_t\right)-\left(1-\chi_{j, t}\right)\left(q_t k_{j, t} / n_{j, t}\right) \sigma_{R, t} \cdot \pi_t+\theta_{j, t} \cdot \pi_t \\
\sigma_{n_j, t} & :=\chi_{j, t}\left(q_t k_{j, t} / n_{j, t}\right) \sigma_{R, t}+\theta_{j, t} \\
\tilde{\sigma}_{n_j, t} & :=\chi_{j, t}\left(q_t k_{j, t} / n_{j, t}\right) \sqrt{\varsigma_t}
\end{aligned}
$$



Households FOC on choices of leverage $q k_h / n_h \geq 0$

$$
\begin{gathered}
\mu_{R, h}-r+\left(1-\gamma_h\right)\left(\sigma_X \sigma_R\right) \cdot \partial_X \ln \xi_h \leq \gamma_h\left(\sigma_{n_h} \cdot \sigma_R+\tilde{\sigma}_{n_h} \sqrt{\varsigma}\right) 
\end{gathered}
$$

Households FOC on market hedges $\theta$

$$
\begin{gathered}
\pi+\left(1-\gamma_h\right) \sigma_X^{\prime} \partial_X \ln \xi_h=\gamma_h \sigma_{n_h} .
\end{gathered}
$$

- Households Euler Equation

  $$
  \left\{\begin{array}{lll}
  \mu_{R, h}-r \leq \pi \cdot \sigma_R, & \text { if } & k_h=0 \\
  \mu_{R, h}-r=\pi \cdot \sigma_R+\gamma_h \varsigma q k_h / n_h, & \text { if } & k_h>0
  \end{array}\right.
  $$

  - Optimal Risk Allocation

    $$
    \begin{aligned}
    \sigma_{n_h} & =\frac{q k_h}{n_h} \sigma_R+\theta_h=\frac{\pi}{\gamma_h}+\frac{1-\gamma_h}{\gamma_h} \sigma_X^{\prime} \partial_X \ln \xi_h \\
    \beta_h & =\frac{q k_h}{n_h} \begin{cases}=\frac{\Delta_h^{+}}{\gamma_h \varsigma}, & \text { if } \varsigma>0 \\
    \geq 0, & \text { if } \varsigma=0 .\end{cases}
    \end{aligned}
    $$

    - gap between the households' expected return on capital and the risk-premium paid by the market

      $\Delta_{h, t}:=\mu_{R, h, t}-r_t-\pi_t \cdot \sigma_{R, t}$





Expert FOC on choices of leverage $q k_e / n_e \geq 0$

$$
\begin{aligned}
& \mu_{R, e}-r-(1-\chi) \pi \cdot \sigma_R+\chi\left(1-\gamma_e\right)\left(\sigma_X \sigma_R\right) \cdot \partial_X \ln \xi_e=\gamma_e \chi\left(\sigma_R \cdot \sigma_{n_e}+\sqrt{\varsigma} \tilde{\sigma}_{n_e}\right) 
\end{aligned}
$$

Experts FOC on equity retention $\chi \geq \underline{\chi}$

$$
\begin{aligned}
\pi \cdot \sigma_R+\left(1-\gamma_e\right)\left(\sigma_X \sigma_R\right) \cdot \partial_X \ln \xi_e \leq \gamma_e\left(\sigma_R \cdot \sigma_{n_e}+\sqrt{\varsigma} \tilde{\sigma}_{n_e}\right),
\end{aligned}
$$

- Expert Euler Equation

  $$
  \begin{cases}\pi \cdot \sigma_R \leq \mu_{R, e}-r, & \text { if } \quad \chi=\underline{\chi} \\ \pi \cdot \sigma_R=\mu_{R, e}-r, & \text { if } \quad \chi>\underline{\chi} .\end{cases}
  $$

  - Optimal Leverage

    $$
    \begin{aligned}
    \beta_e&:=\frac{\chi q k_e}{n_e}=\frac{1}{\gamma_e\left(\left\|\sigma_R\right\|^2+\varsigma\right)}\left[\Delta_e+\pi \cdot \sigma_R+\left(1-\gamma_e\right)\left(\sigma_X \sigma_R\right) \cdot \partial_X \ln \xi_e\right]\\
    \sigma_{n_e}&=\beta_e \sigma_R
    \end{aligned}
    $$

    - wedge between expected return and risk-premium

      $\Delta_{e, t}:=\underline{\chi}^{-1}\left[\mu_{R, e, t}-r_t-\pi_t \cdot \sigma_{R, t}\right]$



Households Value Function

$$
\begin{aligned}
0= & \frac{\psi_h}{1-\psi_h} \rho_h^{1 / \psi_h} \xi_h^{1-1 / \psi_h}-\frac{\rho_h}{1-\psi_h}+r+\frac{1}{2 \gamma_h}\left(\|\pi\|^2+\left(\gamma_h \beta_h \sqrt{\varsigma}\right)^2\right)+\left[\mu_X+\frac{1-\gamma_h}{\gamma_h} \sigma_X \pi\right] \cdot \partial_X \ln \xi_h \\
& +\frac{1}{2}\left[\operatorname{tr}\left(\sigma_X^{\prime} \partial_{X X^{\prime}} \ln \xi_h \sigma_X\right)+\frac{1-\gamma_h}{\gamma_h}\left\|\sigma_X^{\prime} \partial_X \ln \xi_h\right\|^2\right]
\end{aligned}
$$


Experts Value Function

$$
\begin{aligned}
0= & \frac{\psi_h}{1-\psi_h} \rho_h^{1 / \psi_h} \xi_h^{1-1 / \psi_h}-\frac{\rho_h}{1-\psi_h}+r+\frac{1}{2 \gamma_h}\left(\|\pi\|^2+\left(\gamma_h \beta_h \sqrt{\varsigma}\right)^2\right)+\left[\mu_X+\frac{1-\gamma_h}{\gamma_h} \sigma_X \pi\right] \cdot \partial_X \ln \xi_h \\
& +\frac{1}{2}\left[\operatorname{tr}\left(\sigma_X^{\prime} \partial_{X X^{\prime}} \ln \xi_h \sigma_X\right)+\frac{1-\gamma_h}{\gamma_h}\left\|\sigma_X^{\prime} \partial_X \ln \xi_h\right\|^2\right]
\end{aligned}
$$





# Computing Equilibrium

Define
$$
\kappa_t:=\frac{K_{e, t}}{K_t}
$$

$$
w_t:=\int_{\mathbb{J}_e} n_{j, t} d j /\left(q_t K_t\right)
$$

Euler Inequalities

1. $\kappa=1 \text { and } \chi>\underline{\chi}$
   $$
   \mu_{R, e}-r=\pi \cdot \sigma_R>\mu_{R, h}-r
   $$

2. $\kappa=1 \text { and } \chi=\underline{\chi}$
   $$
   \mu_{R, e}-r>\pi \cdot \sigma_R>\mu_{R, h}-r
   $$

3. $\kappa<1 \text { and } \chi>\underline{\chi}$
   $$
   \mu_{R, e}-r>\mu_{R, h}-r \geq \pi \cdot \sigma_R
   $$

Slackness Condition
$$
\begin{aligned}
& 0=\min \left(1-\kappa, \Delta_h^{+}-\Delta_h\right) \\
& 0=\min \left(\chi-\underline{\chi}, \Delta_e\right) \\
& 0=(1-\kappa)(\chi-\underline{\chi})\left(a_e-a_h\right) .
\end{aligned}
$$



Goods Market Condition with optimized investment and consumption 
$$
\int_{\mathbb{J} \in \mathbb{J}_h} c_{j, t} d j=\int_{\mathbb{J}_e \cup \mathbb{J}_h}\left(a_j-\Phi\left(\iota_{j, t}\right)\right) k_{j, t} d j .
$$

$$
c_{j, t}=\rho_j^{1 / \psi_j} \xi_{j, t}^{1-1 / \psi_j} n_{j, t}
$$

Capital Price
$$
q(1-w) \rho_h^{1 / \psi_h} \xi_h^{1-1 / \psi_h}+q w \rho_e^{1 / \psi_e} \xi_e^{1-1 / \psi_e}=(1-\kappa) a_h+\kappa a_e-\Phi(\iota(q))
$$

$$
q=\frac{(1-\kappa) a_h+\kappa a_e+1 / \phi}{(1-w) \rho_h^{1 / \psi_h} \xi_h^{1-1 / \psi_h}+w \rho_e^{1 / \psi_e} \xi_e^{1-1 / \psi_e}+1 / \phi}
$$





# Wealth Share Evolution

Capital Evolution 

$$
\begin{aligned}
\frac{d K_t}{K_t} & =\mu_{K, t} d t+\sigma_{K, t} \cdot d Z_t \\
\mu_{K, t} & :=g_t+\iota\left(q_t\right)-\delta
\end{aligned}
$$

Net Wealth Evolution

$$
\frac{d n_{j, t}}{n_{j, t}}=\left(\mu_{n_j, t}-c_{j, t} / n_{j, t}\right) d t+\sigma_{n_j, t} \cdot d Z_t+\tilde{\sigma}_{n_j, t} d Z_{j, t}
$$

- where

$$
\begin{aligned}
\mu_{n_j, t} & :=r_t+\left(q_t k_{j, t} / n_{j, t}\right)\left(\mu_{R, j, t}-r_t\right)-\left(1-\chi_{j, t}\right)\left(q_t k_{j, t} / n_{j, t}\right) \sigma_{R, t} \cdot \pi_t+\theta_{j, t} \cdot \pi_t \\
\sigma_{n_j, t} & :=\chi_{j, t}\left(q_t k_{j, t} / n_{j, t}\right) \sigma_{R, t}+\theta_{j, t} \\
\tilde{\sigma}_{n_j, t} & :=\chi_{j, t}\left(q_t k_{j, t} / n_{j, t}\right) \sqrt{\varsigma_t}
\end{aligned}
$$

$$
\beta_h:=\frac{q k_h}{n_h}\\
\beta_e:=\frac{\chi q k_e}{n_e}\\
\Delta_{h, t}:=\mu_{R, h, t}-r_t-\pi_t \cdot \sigma_{R, t}\\
\Delta_{e, t}:=\underline{\chi}^{-1}\left[\mu_{R, e, t}-r_t-\pi_t \cdot \sigma_{R, t}\right]
$$

$$
\begin{aligned}
& \frac{d N_{h, t}}{N_{h, t}}=\left[r_t-\rho_h^{1 / \psi_h} \xi_h^{1-1 / \psi_h}-\lambda_d+\sigma_{n_h, t} \cdot \pi_t+\beta_{h, t} \Delta_{h, t}+\frac{(1-\nu) \lambda_d}{1-w_t}\right] d t+\sigma_{n_h, t} \cdot d Z_t \\
& \frac{d N_{e, t}}{N_{e, t}}=\left[r_t-\rho_e^{1 / \psi_e} \xi_e^{1-1 / \psi_e}-\lambda_d+\sigma_{n_e, t} \cdot \pi_t+\beta_{e, t} \Delta_{e, t}+\frac{\nu \lambda_d}{w_t}\right] d t+\sigma_{n_e, t} \cdot d Z_t .
\end{aligned}
$$

Wealth Share Evolution

$$
w=\frac{N_e}{N_e+N_h}
$$

$$
d w=w(1-w)\left[\frac{d N_e}{N_e}-\frac{d N_h}{N_h}\right]-w(1-w)\left[w \frac{d\left[N_e\right]}{N_e^2}-(1-w) \frac{d\left[N_h\right]}{N_h^2}+(1-2 w) \frac{d\left[N_e, N_h\right]}{N_e N_h}\right] .
$$


$$
\int_{\mathbb{J}_e \cup \mathbb{J}_h}\left(1-\chi_{j, t}\right) q_t k_{j, t} \sigma_{R, t} d j=\int_{\mathbb{J}_e \cup \mathbb{J}_h} \theta_{j, t} n_{j, t} d j .
$$

$$
\begin{aligned}
\sigma_{n_h} & =\frac{1-\chi \kappa}{1-w} \sigma_R \\
\sigma_{n_e} & =\frac{\chi \kappa}{w} \sigma_R .
\end{aligned}
$$

$$
\begin{aligned}
\mu_w= & w(1-w)\left[\rho_h^{1 / \psi_h} \xi_h^{1-1 / \psi_h}-\rho_e^{1 / \psi_e} \xi_e^{1-1 / \psi_e}+\beta_e \Delta_e-\beta_h \Delta_h\right] \\
& +(\chi \kappa-w) \sigma_R \cdot\left(\pi-\sigma_R\right)+\lambda_d(\nu-w) \\
\sigma_w= & (\chi \kappa-w) \sigma_R .
\end{aligned}
$$





# Capital Price Evolution

$$
\begin{aligned}
d q\left(X_t\right) & =\left[\mu_X\left(X_t\right) \cdot \partial_X q\left(X_t\right)+\frac{1}{2} \operatorname{tr}\left(\sigma_X\left(X_t\right)^{\prime} \partial_{X X^{\prime}} q\left(X_t\right) \sigma_X\left(X_t\right)\right)\right] d t \\
& +\left[\sigma_X\left(X_t\right)^{\prime} \partial_X q\left(X_t\right)\right] \cdot d Z_t
\end{aligned}
$$

$$
\mu_q=\frac{1}{q}\left[\mu_X \cdot \partial_X q+\frac{1}{2} \operatorname{tr}\left(\sigma_X^{\prime} \partial_{X X^{\prime}} q \sigma_X\right)\right]
$$

$$
\sigma_q=\frac{(\chi \kappa-w)\left(\partial_w \ln q\right) \sqrt{s} \sigma+\left(\partial_g \ln q\right) \sqrt{s} \sigma_g+\left(\partial_s \ln q\right) \sqrt{s} \sigma_s+\left(\partial_{\varsigma} \ln q\right) \sqrt{\varsigma} \sigma_{\varsigma}}{1-(\chi \kappa-w) \partial_w \ln q}
$$




$$
\sigma_R=\sigma_K+\sigma_q
$$

$$
\sigma_R=\frac{\sqrt{s} \sigma+\left(\partial_g \ln q\right) \sqrt{s} \sigma_g+\left(\partial_s \ln q\right) \sqrt{s} \sigma_s+\left(\partial_{\varsigma} \ln q\right) \sqrt{\varsigma} \sigma_{\varsigma}}{1-(\chi \kappa-w) \partial_w \ln q} .
$$





# Interest Rate and Risk Price


$$
\int_{\mathbb{J}_e \cup \mathbb{J}_h}\left(q_t k_{j, t}-n_{j, t}\right) d j=0
$$

$$
\begin{aligned}
r+(1-w)\left(\sigma_{n_h} \cdot \pi+\beta_h \Delta_h-\rho_h^{1 / \psi_h} \xi_h^{1-1 / \psi_h}\right)+w\left(\sigma_{n_e} \cdot \pi+\beta_e \Delta_e\right. & \left.-\rho_e^{1 / \psi_e} \xi_e^{1-1 / \psi_e}\right) \\
& =\mu_q+\mu_K+\sigma_K \cdot \sigma_q
\end{aligned}
$$

$$
(1-w) \sigma_{n_h}+w \sigma_{n_e}=\sigma_R
$$

$$
\begin{aligned}
r=\mu_q & +\mu_K+\sigma_K \cdot \sigma_q-\sigma_R \cdot \pi-(1-w)\left(\beta_h \Delta_h-\rho_h^{1 / \psi_h} \xi_h^{1-1 / \psi_h}\right) \\
& -w\left(\beta_e \Delta_e-\rho_e^{1 / \psi_e} \xi_e^{1-1 / \psi_e}\right) .
\end{aligned}
$$

$$
\pi=\gamma_h \frac{1-\chi \kappa}{1-w} \sigma_R+\left(\gamma_h-1\right) \sigma_X^{\prime} \partial_X \ln \xi_h
$$





# Risk Wedge

$$
\beta_e:=\frac{\chi q k_e}{n_e}=\frac{1}{\gamma_e\left(\left\|\sigma_R\right\|^2+\varsigma\right)}\left[\Delta_e+\pi \cdot \sigma_R+\left(1-\gamma_e\right)\left(\sigma_X \sigma_R\right) \cdot \partial_X \ln \xi_e\right],
$$

$$
\Delta_e=\gamma_e \frac{\chi \kappa}{w}\left(\left\|\sigma_R\right\|^2+\varsigma\right)-\gamma_h \frac{1-\chi \kappa}{1-w}\left\|\sigma_R\right\|^2-\sigma_R^{\prime} \sigma_X^{\prime} \partial_X \ln \left(\frac{\xi_h^{\gamma_h-1}}{\xi_e^{\gamma_e-1}}\right)
$$


$$
\mu_{R, e}-\mu_{R, h}=\mu_{R, e}-r-\pi \cdot \sigma_R+\pi \cdot \sigma_R-\left(\mu_{R, h}-r\right)
$$

$$
\mu_{R, j, t}=\left[\frac{a_j-\Phi\left(\iota_{j, t}\right)}{q_t}+\iota_{j, t}-\delta+g_t+\mu_{q, t}+\sigma_{K, t} \cdot \sigma_{q, t}\right]
$$

$$
\Delta_{e, t}:=\underline{\chi}^{-1}\left[\mu_{R, e, t}-r_t-\pi_t \cdot \sigma_{R, t}\right]
$$

$$
\Delta_{h, t}:=\mu_{R, h, t}-r_t-\pi_t \cdot \sigma_{R, t}
$$

$$
\Delta_h=\underline{\chi} \Delta_e-\frac{a_e-a_h}{q}
$$





# Policy Function


$$
\begin{aligned}
\beta_h & =\frac{q k_h}{n_h} \begin{cases}=\frac{\Delta_h^{+}}{\gamma_h \varsigma}, & \text { if } \varsigma>0 \\
\geq 0, & \text { if } \varsigma=0 .\end{cases}
\end{aligned}
$$

$$
\Delta_h^{+}=\frac{1-\kappa}{1-w} \gamma_h \varsigma
$$




$$
\begin{aligned}
& 0=\min \left(1-\kappa, \Delta_h^{+}-\Delta_h\right)
\end{aligned}
$$

$$
\begin{gathered}
0=\min \left\{1-\kappa, w \gamma_h(1-\chi \kappa)\left\|\sigma_R\right\|^2+w \gamma_h\left(\frac{1-\kappa}{\underline{\chi}}\right) \varsigma-(1-w) \gamma_e \chi \kappa\left(\left\|\sigma_R\right\|^2+\varsigma\right)\right. \\
\left.+w(1-w) \frac{a_e-a_h}{\underline{\chi} q}+w(1-w) \sigma_R^{\prime} \sigma_X^{\prime} \partial_X \ln \left(\frac{\xi_h^{\gamma_h-1}}{\xi_e^{\gamma_e-1}}\right)\right\} .
\end{gathered}
$$



$$
\begin{aligned}
& 0=\min \left(\chi-\underline{\chi}, \Delta_e\right)
\end{aligned}
$$

$$
\begin{aligned}
0=\min \{\chi-\underline{\chi} & ,(1-w) \gamma_e \chi \kappa\left(\left\|\sigma_R\right\|^2+\varsigma\right)-w \gamma_h(1-\chi \kappa)\left\|\sigma_R\right\|^2 \\
& \left.-w(1-w) \sigma_R^{\prime} \sigma_X^{\prime} \partial_X \ln \left(\frac{\xi_h^{\gamma_h-1}}{\xi_e^{\gamma_e-1}}\right)\right\} .
\end{aligned}
$$


$$
\begin{aligned}
& 0=(1-\kappa)(\chi-\underline{\chi})\left(a_e-a_h\right) .
\end{aligned}
$$

$$
\begin{aligned}
0= & \min \left\{\chi-\underline{\chi},(1-w) \gamma_e \varsigma\left(\partial_w \ln q\right)^2(\chi-w)^3+(1-w) \gamma_e \varsigma\left(\partial_w \ln q\right)\left(w \partial_w \ln q-2\right)(\chi-w)^2\right. \\
& +\left[\left((1-w) \gamma_e+w \gamma_h\right)\left\|D_{\tilde{X}}\right\|^2+(1-w) \gamma_e \varsigma\left(1-2 w \partial_w \ln q\right)+\left(\partial_w \ln q\right) D_{\xi, \tilde{X}}-D_{\xi, w}\right](\chi-w) \\
& \left.+w(1-w)\left(\gamma_e-\gamma_h\right)\left\|D_{\tilde{X}}\right\|^2+w(1-w) \gamma_e \varsigma-D_{\xi, \tilde{X}}\right\} .
\end{aligned}
$$

$$
\begin{aligned}
0= & \min \left\{\chi-\underline{\chi},(1-w) \gamma_e \varsigma\left(\partial_w \ln q\right)^2(\chi-w)^3+(1-w) \gamma_e \varsigma\left(\partial_w \ln q\right)\left(w \partial_w \ln q-2\right)(\chi-w)^2\right. \\
& +\left[\left((1-w) \gamma_e+w \gamma_h\right)\left\|D_{\tilde{X}}\right\|^2+(1-w) \gamma_e \varsigma\left(1-2 w \partial_w \ln q\right)+\left(\partial_w \ln q\right) D_{\xi, \tilde{X}}-D_{\xi, w}\right](\chi-w) \\
& \left.+w(1-w)\left(\gamma_e-\gamma_h\right)\left\|D_{\tilde{X}}\right\|^2+w(1-w) \gamma_e \varsigma-D_{\xi, \tilde{X}}\right\} .
\end{aligned}
$$

$$
\begin{aligned}
\sigma_{\tilde{X}} & :=\left(\sqrt{s} \sigma_g, \sqrt{s} \sigma_s, \sqrt{\varsigma} \sigma_{\varsigma}\right)^{\prime} \\
D_{\tilde{X}} & :=\sigma_K+\sigma_{\tilde{X}}^{\prime} \partial_{\tilde{X}} \ln q \\
D_{\xi, w} & :=w(1-w)\left\|D_{\tilde{X}}\right\|^2 \partial_w\left[\ln \left(\xi_h^{\gamma_h-1}\right)-\ln \left(\xi_e^{\gamma_e-1}\right)\right] \\
D_{\xi, \tilde{X}} & :=w(1-w)\left(\sigma_{\tilde{X}} D_{\tilde{X}}\right) \cdot \partial_{\tilde{X}}\left[\ln \left(\xi_h^{\gamma_h-1}\right)-\ln \left(\xi_e^{\gamma_e-1}\right)\right] .
\end{aligned}
$$



In [ ]:
dataname = "test"
params = m.paramsDefault.copy()

## Dimensionality params
params['nDims']             = 3
params['nShocks']           = 3

## Grid parameters 
params['numSds']            = 5
params['uselogW']           = 0

params['nWealth']           = 100
params['nZ']                = 30
params['nV']                = 30
params['nVtilde']           = 0

## Economic params
params['nu_newborn']        = 0.1
params['lambda_d']          = 0.02
params['lambda_Z']          = 0.252
params['lambda_V']          = 0.156
params['lambda_Vtilde']     = 1.38
params['delta_e']           = 0.05
params['delta_h']           = 0.05
params['a_e']               = 0.14
params['a_h']               = 0.135
params['rho_e']             = 1.0
params['rho_h']             = 1.0
params['phi']               = 3.0
params['gamma_e']           = 1.0
params['gamma_h']           = 1.0
params['equityIss']         = 2
params['chiUnderline']      = 1.0
params['alpha_K']           = 0.05

## Alogirthm behavior and results savings params
params['method']            = 2
params['dt']                = 0.1
params['dtInner']           = 0.1

params['tol']               = 1e-5
params['innerTol']          = 1e-5

params['verbatim']          = -1
params['maxIters']          = 4000
params['maxItersInner']     = 2000000
params['iparm_2']           = 28
params['iparm_3']           = 0
params['iparm_28']          = 0
params['iparm_31']          = 0
params['overwrite']         = 'Yes'
params['exportFreq']        = 10000
params['CGscale']           = 1.0
params['hhCap']             = 1
params['preLoad']           = 'None'

# Domain params
params['Vtilde_bar']        = 0.0
params['Z_bar']             = 0.0
params['V_bar']             = 1.0
params['sigma_K_norm']      = 0.04
params['sigma_Z_norm']      = 0.0141
params['sigma_V_norm']      = 0.132
params['sigma_Vtilde_norm'] = 0.0
params['wMin']              = 0.01
params['wMax']              = 0.99

## Shock correlation params
params['cov11']             = 1.0
params['cov12']             = 0.0
params['cov13']             = 0.0
params['cov14']             = 0.0
params['cov21']             = 0.0
params['cov22']             = 1.0
params['cov23']             = 0.0
params['cov24']             = 0.0
params['cov31']             = 0.0
params['cov32']             = 0.0
params['cov33']             = 1.0
params['cov34']             = 0.0
params['cov41']             = 0.0
params['cov42']             = 0.0
params['cov43']             = 0.0
params['cov44']             = 0.0

psi_e = str("{:0.3f}".format(params['rho_e'])).replace('.', '', 1) 
psi_h = str("{:0.3f}".format(params['rho_h'])).replace('.', '', 1) 
gamma_e = str("{:0.3f}".format(params['gamma_e'])).replace('.', '', 1) 
gamma_h = str("{:0.3f}".format(params['gamma_h'])).replace('.', '', 1) 
a_e = str("{:0.3f}".format(params['a_e'])).replace('.', '', 1) 
a_h = str("{:0.3f}".format(params['a_h'])).replace('.', '', 1) 
chiUnderline = str("{:0.3f}".format(params['chiUnderline'])).replace('.', '', 1) 

if params['nVtilde'] == 0.0:
    folder_name = 'output/'+dataname+'/WZV_chiUnderline_' + chiUnderline + '_a_e_' + a_e + '_a_h_' + a_h  + '_gamma_e_' + gamma_e + '_gamma_h_' + gamma_h + '_psi_e_' + psi_e + '_psi_h_' + psi_h
elif params['nV'] == 0.0:
    folder_name = 'output/'+dataname+'/WZVtilde_chiUnderline_' + chiUnderline + '_a_e_' + a_e + '_a_h_' + a_h  + '_gamma_e_' + gamma_e + '_gamma_h_' + gamma_h + '_psi_e_' + psi_e + '_psi_h_' + psi_h
else:
    print('Error: either nV or nVtilde must be zero')

# params['preLoad']          = folder_name 
params['folderName']        = folder_name

#### Now, create a Model
Model = m.Model(params)

In [ ]:
# Step 2: Solve the model
#------------------------------------------#

#### This step is very simple: use the .solve() method.
start = time.time()
Model.solve()
Model.printInfo() ## This step is optional: it prints out information regarding time, number of iterations, etc.
Model.printParams() ## This step is optional: it prints out the parameteres used.
Model.computeStatDent() # Compustes the stationary density of the model.
Model.dumpData()
end = time.time()
solve_time = '{:.4f}'.format((end - start)/60)
MFR_time_info = {'solve_time': solve_time}
with open(os.getcwd()+"/" + folder_name + "/MFR_time_info.json", "w") as f:
    json.dump(MFR_time_info,f)

# %%
start = time.time()

Model.computeShockElas(pcts = {'W':[.5], 'Z': [0.5], 'V': [0.25, 0.5, 0.75]}, T = 48, dt = 1, perturb = 'Ce', bc = {'natural': True})
with open(os.getcwd()+"/" + folder_name + "/ExpertsExpoConsumption.pkl", 'wb') as file:   
    pickle.dump(Model.expoElas, file)
with open(os.getcwd()+"/" + folder_name + "/ExpertsPriceConsumption.pkl", 'wb') as file:   
    pickle.dump(Model.priceElasExperts, file)
Model.computeShockElas(pcts = {'W':[.5], 'Z': [0.5], 'V': [0.25, 0.5, 0.75]}, T = 48, dt = 1, perturb = 'Ch', bc = {'natural': True})
with open(os.getcwd()+"/" + folder_name + "/HouseholdsExpoConsumption.pkl", 'wb') as file:   
    pickle.dump(Model.expoElas, file)
with open(os.getcwd()+"/" + folder_name + "/HouseholdsPriceConsumption.pkl", 'wb') as file:   
    pickle.dump(Model.priceElasHouseholds, file)
end = time.time()
solve_time = '{:.4f}'.format((end - start)/60)
Ela_time_info = {'solve_time': solve_time}
with open(os.getcwd()+"/" + folder_name + "/Ela_time_info.json", "w") as f:
    json.dump(Ela_time_info,f)
Model.dumpData()


In [ ]:
# %%
##### This method can only be called after the model is solved.
pcts = {'W':[.5],'Z':[.5],'V':[.25,.5,.75]}

# 48 year time periods
T = 48
dt = 1

# Natural boundatry conditions
bc = {'natural':True}

# Use defaults starting points
points = np.matrix([])

## Create input stateMat for shock elasticities, a tuple of ranges of the state space
Model.stateMatInput = []

for i in range(Model.params['nDims']):
    ## Note that i starts at zero but our variable names start at 1.
    Model.stateMatInput.append(np.linspace(np.min(Model.stateMat.iloc[:,i]),
                                np.max(Model.stateMat.iloc[:,i]),
                                np.unique(np.array(Model.stateMat.iloc[:,i])).shape[0]) )
## Create dictionary to store model
if Model.model is None:
    Model.model = {}

allPcts = []

## Find points
if points.shape[1] == 0:
    allPts = []
    for stateVar in Model.stateVarList:
        if Model.dent is None or np.max(Model.dent) < 0.0001:
            raise Exception("Stationary density not computed or degenerate.")
        allPts.append([Model.inverseCDFs[stateVar](pct) for pct in pcts[stateVar]])
        allPcts.append(pcts[stateVar])
    Model.x0 = np.matrix(list(itertools.product(*allPts)))
    allPcts = [list(x) for x in list(itertools.product(*allPcts))]
    Model.pcts = pcts
else:
    Model.x0 = points

modelsol = {
    'dent' : Model.dent,
    'muCe' : Model.muCe(),
    'muCh' : Model.muCh(),
    'muSe' : Model.muSe(),
    'muSh' : Model.muSh(),
    'muX'  : Model.muX(),
    'muNh'  : -0.5*np.sum([((1-Model.params['gamma_h'])/(Model.params['rho_h']-Model.params['gamma_h'])*\
                                    (Model.sigmaSh()[:,s]+\
                                     Model.params['rho_h']*Model.sigmaCh()[:,s]))**2\
                                    for s in range(Model.params['nDims'])], axis = 0),
    'muNe'  : -0.5*np.sum([((1-Model.params['gamma_e'])/(Model.params['rho_e']-Model.params['gamma_e'])*\
                                    (Model.sigmaSe()[:,s]+\
                                     Model.params['rho_e']*Model.sigmaCe()[:,s]))**2\
                                    for s in range(Model.params['nDims'])], axis = 0),
    'sigmaSe' : Model.sigmaSe(),
    'sigmaSh' : Model.sigmaSh(),
    'sigmaCe' : Model.sigmaCe(),
    'sigmaCh' : Model.sigmaCh(),
    'sigmaX'  : Model.sigmaXList,
    'sigmaNh' : (1-Model.params['gamma_h'])/(Model.params['rho_h']-Model.params['gamma_h'])*\
                                    (Model.sigmaSh()+\
                                     Model.params['rho_h']*Model.sigmaCh()),
    'sigmaNe' : (1-Model.params['gamma_e'])/(Model.params['rho_e']-Model.params['gamma_e'])*\
                                    (Model.sigmaSe()+\
                                     Model.params['rho_e']*Model.sigmaCe()),
    'stateMatInput' : Model.stateMatInput,
    'gridSizeList' : Model.gridSizeList,
    'x0' : Model.x0,
    'nDims' : Model.params['nDims'],
    'nShocks' : Model.params['nShocks']
}

with open(os.getcwd()+"/" + folder_name + "/model_ela_preload.pkl", "wb") as f:
    pickle.dump(modelsol,f)